In [1]:
from numba import cuda
import numpy as np
import math

## Data loader

In [14]:
def read_data(fn):
    with open(fn, "r") as fh:
        data = []
        for line in fh.readlines():
            if not line:
                continue
            direction, value = line.strip().split(" ")
            data.append([ord(direction[0]), int(value)])
        return np.asarray(data)

In [15]:
read_data("sample.txt")

array([[102,   5],
       [100,   5],
       [102,   8],
       [117,   3],
       [100,   8],
       [102,   2]])

## Part 1

In [17]:
@cuda.jit
def update_position(in_arr, output):
    pos = cuda.grid(1)
    if pos < in_arr.size:
        # Down
        if in_arr[pos][0] == 100:
            cuda.atomic.add(output, 1, in_arr[pos][1])
        # Up
        if in_arr[pos][0] == 117:
            cuda.atomic.add(output, 1, -in_arr[pos][1])
        # Forward
        if in_arr[pos][0] == 102:
            cuda.atomic.add(output, 0, in_arr[pos][1])

In [22]:
def run_part1(arr):
    # Create output array
    output_arr = np.zeros(2)
    
    # Configure grid
    threads = 128
    blocks = math.ceil(len(arr) / threads)
    
    # Calculate increasing/decreasing
    update_position[blocks, threads](arr, output_arr)
    
    # Return answer
    return int(output_arr.prod())

In [23]:
run_part1(read_data("sample.txt"))

150

In [24]:
run_part1(read_data("input.txt"))

2073315

## Part 2

Given that each item in the array needs the output from the previous item this will not parallelise.